# Register with DIVD

Register the NLM Hashes from the LEAK specific database into the shared DIVD leak DB

# Set constants

In [ ]:
CASE="DIVD-2021-00004"
SUB="logs"
DESCRIPTION="Office phishing campaign"
CASE_DATE="2020-04-09"

# Keep these the same
IN_DIR="../LEAK_DB/"
IN_DB=f"{IN_DIR}/{CASE}-{SUB}.sqlite3"
DIVD_DB="../SHARED_DBS/DIVD-leaks.sqlite3"

# Import helper functions

In [ ]:
%run ../0.shared_notebooks/0_helper_functions.ipynb

# Open DB

In [ ]:
divd_conn=sqlite3.connect(DIVD_DB)
divd_cur=divd_conn.cursor()

In [ ]:
divd_conn


# Check if leak exists

In [ ]:
divd_cur.execute( "SELECT count(*) FROM leak WHERE case_id = ? and sub_id = ?;",  ( CASE, SUB ) )
row = divd_cur.fetchone()
if row[0] > 0 :
    print("Leak exists", file=sys.stderr)
else:
    print("Leak is new")

# Import leak

In [ ]:
divd_conn.execute( 
    "INSERT INTO leak ( case_id, sub_id, description, date) values (?, ?, ?, ?);", 
    ( CASE, SUB, DESCRIPTION, CASE_DATE ) 
)
divd_conn.commit()

# Import hashes

In [ ]:
in_conn=sqlite3.connect(IN_DB)

In [ ]:
in_conn

In [ ]:
divd_conn.execute("DELETE FROM leak WHERE case_id = ? and sub_id = ?;", ( CASE, SUB ) )
count = 0
for row in in_conn.execute("SELECT DISTINCT nml_hash FROM entity").fetchall() :
    if row[0] :
        divd_conn.execute(
            "INSERT INTO divd ( 'nml_hash', 'case', 'sub_id' ) values ( ?, ?, ?);",
            ( row[0], CASE, SUB )
        )
        count=count+1
        if count % 1000 == 0 :
            print(f"{count} \r")
        if count % 10_000 == 0 :
            divd_conn.commit()
print(count)
divd_conn.commit()